In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr

2025-07-16 10:40:21.663714: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-16 10:40:21.699662: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-16 10:40:21.699686: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-16 10:40:21.700614: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-16 10:40:21.706688: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# I think we can get the metrics in a for loop for all the blocks individually?
# And later we will compute an average metric too

In [3]:
# get teh trained model?
non_overlapping_model = tf.keras.models.load_model("models/CNN_seq2seq_non_overlapping_lieanr_activation.keras")

2025-07-16 10:40:34.938287: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


In [4]:
# locate the test data

# input features
input_features_loc = 'data/test_input_sub_images'
input_contents = os.listdir(input_features_loc)
input_contents.sort()

In [5]:
# test targets
out_targets_loc = 'data/test_out_targets'
out_contents = os.listdir(out_targets_loc)
out_contents.sort()

In [6]:
# out_contents

In [7]:
%%time
# now how to proceed?

test_rmse = []
test_mae = []
test_r2 = []
test_personr = []
preds = []
for i in range(len(input_contents)):
    # load the features
    test_features = np.load(os.path.join(input_features_loc, input_contents[i]))
    # load targets
    test_targets = np.load(os.path.join(out_targets_loc, out_contents[i]))
    # predict the targets with trained model
    test_preds = non_overlapping_model.predict(test_features)
    preds.append(test_preds)
    # compute the test scores, I think we need to flatten these before computing the scores - or can use tf, but the answers are going to be the same
    test_preds_flatten = test_preds.flatten()
    test_targets_flatten = test_targets.flatten()
    mae = mean_absolute_error(test_targets_flatten, test_preds_flatten)
    test_mae.append(mae)
    rmse = np.sqrt(mean_squared_error(test_targets_flatten, test_preds_flatten))
    test_rmse.append(rmse)
    rsquare = r2_score(test_targets_flatten, test_preds_flatten)
    test_r2.append(rsquare)
    pearsonr_score = pearsonr(test_targets_flatten, test_preds_flatten)[0]
    test_personr.append(pearsonr_score)

2025-07-16 10:40:39.244658: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


29/29 [==============================] - 0s 9ms/step
CPU times: user 9.13 s, sys: 17.6 s, total: 26.7 s
Wall time: 25.7 s


In [8]:
# test_preds.shape

In [9]:
# preds[0].shape

In [10]:
# Make sure to also have a train script without relu activation

In [11]:
test_rmse

[0.30879962,
 0.29384685,
 0.30271357,
 0.28649348,
 0.31647494,
 0.31002837,
 0.29257333,
 0.34608877,
 0.31792518,
 0.29878563,
 0.29929647,
 0.33020166,
 0.32074192]

In [12]:
test_mae

[0.15305659,
 0.14100249,
 0.14740227,
 0.13761362,
 0.15544976,
 0.15270302,
 0.14009817,
 0.168351,
 0.15441547,
 0.14655411,
 0.1458992,
 0.16256148,
 0.15535575]

In [13]:
test_r2

[0.7562358374298113,
 0.8046547479405115,
 0.785569700342231,
 0.8088536284072497,
 0.7457928946855847,
 0.7292214785123414,
 0.8077365103375589,
 0.7007696648695743,
 0.7416432545483644,
 0.7592234762852654,
 0.7758211094295042,
 0.7223086968203993,
 0.7438899624211441]

In [14]:
test_personr

[0.8704956276331984,
 0.9004747300327962,
 0.891740510396194,
 0.9030251362622954,
 0.8652338531570347,
 0.8540719271059278,
 0.9023421577881304,
 0.8372512743623876,
 0.8619038268910867,
 0.871407344458952,
 0.881937041996196,
 0.850417683765101,
 0.8628371690252757]

In [15]:
# Now, do this for all test data at once?

In [16]:
# stack all input and outputs

In [17]:
all_test_features_list = [np.load(os.path.join(input_features_loc, file)) for file in input_contents]

In [18]:
# all_test_features_list[0].shape

In [19]:
# len(all_test_features_list)

In [20]:
# stack all data together
all_test_features = np.vstack(all_test_features_list)

In [21]:
all_test_features.shape

(11830, 13, 30, 30, 3)

In [22]:
all_test_targets_list = [np.load(os.path.join(out_targets_loc, file)) for file in out_contents]

In [23]:
# all_test_targets_list[0].shape

In [24]:
# len(all_test_targets_list)

In [25]:
# stack all data together
all_test_targets = np.vstack(all_test_targets_list)

In [26]:
all_test_targets.shape

(11830, 7, 32)

In [27]:
# get all test predictions
all_test_preds = non_overlapping_model.predict(all_test_features)

370/370 [==============================] - 3s 9ms/step


In [28]:
all_test_preds.shape

(11830, 7, 32)

In [29]:
# flatten these vectors
all_test_targets_flatten = all_test_targets.flatten()
all_test_preds_flatten = all_test_preds.flatten()

In [30]:
all_test_targets_flatten.shape, all_test_preds_flatten.shape

((2649920,), (2649920,))

In [31]:
# compute metrics
all_test_rmse = np.sqrt(mean_squared_error(all_test_targets_flatten, all_test_preds_flatten))
all_test_mae = mean_absolute_error(all_test_targets_flatten, all_test_preds_flatten)
all_test_r2_score = r2_score(all_test_targets_flatten, all_test_preds_flatten)
all_test_pearsonr = pearsonr(all_test_targets_flatten, all_test_preds_flatten)

In [32]:
all_test_rmse, all_test_mae, all_test_r2_score, all_test_pearsonr[0]

(0.30995086, 0.15080488, 0.7619529685452918, 0.8741279399719215)